# Reset the System
The commands in this section will reset the system so that you can run the labs again. 

## Terminal Window
Before running the examples in this lab, you will need to open up a terminal window. If you have not already done so, see the <a href="./Terminal.ipynb">Opening up a Terminal Window</a> section. 

The commands that you need to copy and execute in the terminal window are surrounded with a blue box. Select all the text in the box and then use the copy keys (^C or &#x2318;C) to place the text into the clipboard.

<div class="boxed">The command you need to copy and paste into the terminal window are found in here.
</div>

When you switch to the terminal window, you will use the appropriate Paste command (^&#x21E7;v or &#x2318;v). This will load the command into the terminal window and immediately execute it.

## Reset Trading Database for HADR example
In your terminal window, make sure that you are not current connected as `db2inst1`. To check, run the following command to close the connect if it is not `k8s`.

<div class="boxed">[ $(whoami) != 'k8s' ] && exit</div>

Stop HADR connection on TRADING database

<div class="boxed">clear
ssh db2inst1@10.0.0.1 /bin/bash << EOF
db2 stop hadr on database trading
db2 update db cfg for trading using HADR_LOCAL_HOST NULL
db2 update db cfg for trading using HADR_LOCAL_SVC NULL
db2 update db cfg for trading using HADR_REMOTE_HOST NULL
db2 update db cfg for trading using HADR_REMOTE_SVC NULL
db2 update db cfg for trading using HADR_REMOTE_INST NULL
db2 update db config for trading using logarchmeth1 OFF
db2 update db config for trading using logindexbuild OFF
db2 connect to trading
db2 drop table custcopy
db2 connect reset 
db2stop force
db2start
rm -rf /home/db2inst1/logfiles    
EOF
</div>

Remove the TRADING database from the second site.

<div class="boxed">clear
ssh db2inst1@10.0.0.2 /bin/bash << EOF    
db2 connect reset
db2 deactivate database trading
db2 drop database trading
EOF
</div>

## Reset the Kubernetes Service
We need to drop the Db2U Pods and then the Db2 Operator from the Kubernetes system. Hit CTRL-C to stop the command when all of the pods are gone.

<div class="boxed">clear
kubectl delete -f ~/Documents/db2create.yaml
kubectl get pods -n db2u -w
</div>

You should see output similar to the following:
```
NAME                                    READY   STATUS      RESTARTS   AGE
c-demo-instdb-dnt74                     0/1     Completed   0          2d20h
c-demo-restore-morph-4phbd              0/1     Completed   0          2d20h
db2u-operator-manager-c849d76c4-zvdsm   1/1     Running     2          2d20h
c-demo-etcd-0                           1/1     Running     2          2d20h
c-demo-db2u-0                           1/1     Running     2          2d20h
c-demo-restore-morph-4phbd              0/1     Terminating   0          2d20h
c-demo-restore-morph-4phbd              0/1     Terminating   0          2d20h
c-demo-db2u-0                           1/1     Terminating   2          2d20h
c-demo-etcd-0                           1/1     Terminating   2          2d20h
c-demo-instdb-dnt74                     0/1     Terminating   0          2d20h
c-demo-instdb-dnt74                     0/1     Terminating   0          2d20h
c-demo-etcd-0                           0/1     Terminating   2          2d20h
c-demo-db2u-0                           0/1     Terminating   2          2d20h
```

When it appears there is no more activity, hit CTRL-C. Next get pods again and you should only see the `db2u-operator` running.

<div class="boxed">clear
kubectl get pods -n db2u
</div>

Once you see the following, you can precede to stopping the Db2 Operator service.
```
NAME                                    READY   STATUS    RESTARTS   AGE
db2u-operator-manager-c849d76c4-zvdsm   1/1     Running   2          2d20h
```

<div class="boxed">clear
kubectl delete -f ~/Documents/db2uoperator.yaml
</div>

Once the final subscription operator is deleted, the system will be back to its original state.
```
namespace "db2u" deleted
catalogsource.operators.coreos.com "db2u-catalog" deleted
operatorgroup.operators.coreos.com "db2u-oprator-og" deleted
subscription.operators.coreos.com "db2u-operator-v2-0-0-sub" deleted
```

## Reset Db2 Shift Settings
This section will remove the settings that were saved during the Db2 Shift sessions.

<div class="boxed">clear
ssh db2inst1@10.0.0.1 rm /home/db2inst1/c2c/db2shift.defaults
ssh db2inst2@10.0.0.1 rm /home/db2inst2/c2c/db2shift.defaults
</div>

#### Credits: IBM 2024, George Baklarz [baklarz@ca.ibm.com]